In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
url = "https://editorial.rottentomatoes.com/guide/oscars-best-and-worst-best-pictures/"
r = requests.get(url)

In [3]:
text = r.text
soup = BeautifulSoup(r.text, 'html.parser')

In [4]:
div_tags = soup.find_all('div', {'class': 'article_movie_title'})

In [5]:
titles = []
for div in div_tags:
    a_tags = div.find_all('a')
    for a in a_tags:
        titles.append(a.get_text().strip())

In [6]:
year_tag = soup.find_all("span", {"class":"subtle start-year"})

years = []
for entry in year_tag:
    year = entry.get_text().strip()
    year = year.replace("(","").replace(")","")
    year = int(year)
    years.append(year)

In [7]:
director_tags = soup.find_all('div', {'class': 'info director'})

In [8]:
directors = []
for d in director_tags:
    directors.append(d.get_text().strip().replace('Directed By:', '').strip())

In [9]:
actor_tags = soup.find_all('div', {'class': 'info cast'})

In [10]:
names = []
for actor in actor_tags:
    a_tags = a.find_all('a')
    for a in a_tags:
        names.append(a.get_text().strip())
    
    # name = a.get_text().strip().replace('Starring:', '').strip()
    # names.append(name)
    # for n in names:
    #     print(n)

In [11]:
actors = []
for a in actor_tags:
    a_tags = a.find_all('a')
    for b in a_tags:
        actors.append(b.get_text().strip())

In [15]:
actors

['Song Kang-ho',
 'Lee Sun-kyun',
 'Jo Yeo-jeong',
 'Choi Woo-sik',
 'Humphrey Bogart',
 'Ingrid Bergman',
 'Paul Henreid',
 'Claude Rains',
 'Marlon Brando',
 'Eva Marie Saint',
 'Karl Malden',
 'Lee J. Cobb',
 'Bette Davis',
 'Anne Baxter',
 'Celeste Holm',
 'George Sanders',
 'Mahershala Ali',
 'Alex R. Hibbert',
 'Ashton Sanders',
 'Trevante Rhodes',
 'Liam Neeson',
 'Ben Kingsley',
 'Ralph Fiennes',
 'Caroline Goodall',
 'Claudette Colbert',
 'Clark Gable',
 'Walter Connolly',
 'Roscoe Karns',
 'Laurence Olivier',
 'Joan Fontaine',
 'Judith Anderson',
 'George Sanders',
 'Lew Ayres',
 'Louis Wolheim',
 'John Wray',
 'Raymond Griffith',
 "George O'Brien",
 'Janet Gaynor',
 'Margaret Livingston',
 'Bodil Rosing',
 'Mark Ruffalo',
 'Michael Keaton',
 'Rachel McAdams',
 'Liev Schreiber',
 'Marlon Brando',
 'Al Pacino',
 'James Caan',
 'Richard S. Castellano',
 'Woody Allen',
 'Diane Keaton',
 'Tony Roberts',
 'Carol Kane',
 'Fredric March',
 'Myrna Loy',
 'Dana Andrews',
 'Teresa Wrig

In [14]:
film_data = pd.DataFrame({'Movie Title': titles,
              'Year Released': years,
              'Director': directors})
film_data

,Movie Title,Year Released,Director
0,Parasite,2019,Bong Joon Ho
1,Casablanca,1942,Michael Curtiz
2,On the Waterfront,1954,Elia Kazan
3,All About Eve,1950,Joseph L. Mankiewicz
4,Moonlight,2016,Barry Jenkins
...,...,...,...
93,Cavalcade,1933,Frank Lloyd
94,Out of Africa,1985,Sydney Pollack
95,Cimarron,1931,Wesley Ruggles
96,The Greatest Show on Earth,1952,Cecil B. DeMille
